VAR MODEL with London Bikes Data
Inspired from https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar 
%matplotlib inline
import seaborn as sns
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
bikes = pd.read_csv('london_merged.csv')
bikes.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [3]:
#convert the timestamp column from object to Date format
bikes['timestamp'] = pd.to_datetime(bikes['timestamp'])
bikes.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [4]:
bike_data = bikes.drop(['timestamp'], axis=1)
bike_data.index = bikes.timestamp
bike_data.head()

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
timestamp,,,,,,,,,
2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [5]:
#Check stationarity of the data before applying the VAR model
from statsmodels.tsa.vector_ar.vecm import coint_johansen
coint_johansen(bike_data,-1,1).eig

array([2.61219379e-01, 1.31970167e-01, 5.22046139e-02, 4.19830465e-02,
       2.10126207e-02, 1.75450605e-02, 1.36518877e-02, 6.26085775e-04,
       7.56291478e-05])

In [42]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(bike_data, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(bike_data,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(bike_data.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(bike_data)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
cnt    ::  8197.73   > 179.5199  =>   True
t1     ::  3938.67   > 143.6691  =>   True
t2     ::  2416.01   > 111.7797  =>   True
hum    ::  1490.3    > 83.9383   =>   True
wind_speed ::  899.67    > 60.0627   =>   True
weather_code ::  494.11    > 40.1749   =>   True
is_holiday ::  210.88    > 24.2761   =>   True
is_weekend ::  12.84     > 12.3212   =>   True
season ::  1.2       > 4.1296    =>   False


In [6]:
#splitting the data into train set and test set
#forecasting 4 observations
num_obs = 4
bike_train, bike_test = bike_data[0:-num_obs],bike_data[-num_obs:]

#check the size of the split data
print(bike_train.shape)
print(bike_test.shape)

(17410, 9)
(4, 9)


In [7]:
#Inspired from https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/
#check for stationarity
from statsmodels.tsa.stattools import adfuller
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")   

In [8]:
#testing the data for stationarity
for name, col in bike_train.iteritems():
    adfuller_test(col,name=col.name)
    print('\n')

    Augmented Dickey-Fuller Test on "cnt" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -9.8753
 No. Lags Chosen       = 41
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.9396
 No. Lags Chosen       = 44
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0018. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t2" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05


In [14]:
#Difference the data to make the season column stationary
# 1st difference
bike_differenced = bike_train.diff().dropna()

In [15]:
#Run the adfuller test again
#testing the data for stationarity
for name, col in bike_train.iteritems():
    adfuller_test(col,name=col.name)
    print('\n')

    Augmented Dickey-Fuller Test on "cnt" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -9.8753
 No. Lags Chosen       = 41
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.9396
 No. Lags Chosen       = 44
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0018. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t2" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05


In [16]:
#second differencing
bike_differenced = bike_differenced.diff().dropna()

In [17]:
#Run the adfuller test on the second differencing
#testing the data for stationarity
for name, col in bike_train.iteritems():
    adfuller_test(col,name=col.name)
    print('\n')

    Augmented Dickey-Fuller Test on "cnt" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -9.8753
 No. Lags Chosen       = 41
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.9396
 No. Lags Chosen       = 44
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0018. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "t2" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05


In [25]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(endog=bike_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    model_fit = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', model_fit.aic)
    print('BIC : ', model_fit.bic)
    print('FPE : ', model_fit.fpe)
    print('HQIC: ', model_fit.hqic, '\n')

Lag Order = 1
AIC :  2.5863002989852593
BIC :  2.6264460158396132
FPE :  13.280546563917104
HQIC:  2.5995235975912014 

Lag Order = 2
AIC :  1.4804587682530674
BIC :  1.55673944809466
FPE :  4.394961521591375
HQIC:  1.5055843634291235 

Lag Order = 3
AIC :  0.8765175946091797
BIC :  0.9889368549172449
FPE :  2.402518638012688
HQIC:  0.9135467445086651 

Lag Order = 4
AIC :  0.48434108640480633
BIC :  0.6329025452360468
FPE :  1.6231052639285992
HQIC:  0.5332750493875021 

Lag Order = 5
AIC :  0.26740792708888694
BIC :  0.45211520307759856
FPE :  1.3065734690613309
HQIC:  0.32824796172108556 

Lag Order = 6
AIC :  0.08238505874194935
BIC :  0.303241771100147
FPE :  1.0858740602539003
HQIC:  0.15513242379650002 

Lag Order = 7
AIC :  -0.07175860970024946
BIC :  0.18525115881729404
FPE :  0.9307558219042531
HQIC:  0.012897344756105572 

Lag Order = 8
AIC :  -0.47835625907360013
BIC :  -0.18518981402888052
FPE :  0.6198016224087665
HQIC:  -0.3817904560293399 

Lag Order = 9
AIC :  -0.59635

In [26]:
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,5.083,5.087,161.3,5.085
1,2.587,2.627,13.29,2.600
2,1.481,1.557,4.396,1.506
3,0.8779,0.9903,2.406,0.9149
4,0.4856,0.6342,1.625,0.5346
5,0.2694,0.4542,1.309,0.3303
6,0.08378,0.3047,1.087,0.1566
7,-0.07063,0.1864,0.9318,0.01404
8,-0.4775,-0.1843,0.6203,-0.3809
9,-0.5957,-0.2663,0.5512,-0.4872


In [27]:
#train the model
var_model = model.fit(5)
var_model.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 16, Dec, 2019
Time:                     12:49:02
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                   0.452115
Nobs:                     17403.0    HQIC:                  0.328248
Log likelihood:          -224157.    FPE:                    1.30657
AIC:                     0.267408    Det(Omega_mle):         1.27590
--------------------------------------------------------------------
Results for equation cnt
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                  -0.010797         5.526812           -0.002           0.998
L1.cnt                 -0.427351         0.007549          -56.607           0.000
L1.t1                  29.633351        16.548539     

In [28]:
#forecasting the bike data using the VAR model
#get the number of lags which represents the terms in the VAR model
lag_number = var_model.k_ar
print(lag_number)

#data for forecasting
bike_input = bike_differenced.values[-lag_number:]
bike_input

5


array([[ 1.930e+02,  0.000e+00,  0.000e+00, -5.000e+00,  6.000e+00,
         2.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
       [ 2.760e+02,  0.000e+00,  0.000e+00,  2.500e+00, -6.000e+00,
        -1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.185e+03,  0.000e+00,  0.000e+00,  2.500e+00, -4.000e+00,
        -1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
       [-2.063e+03, -1.000e+00, -1.000e+00,  5.000e+00,  6.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
       [-6.560e+02,  1.000e+00,  1.000e+00, -7.500e+00, -4.000e+00,
         2.000e+00,  0.000e+00,  0.000e+00,  0.000e+00]])

In [29]:
# Forecast the bike data
bike_foc = var_model.forecast(y=bike_input, steps=num_obs)
bike_forecast = pd.DataFrame(bike_foc, index=bike_data.index[-num_obs:], columns=bike_data.columns + '_2d')
bike_forecast

,cnt_2d,t1_2d,t2_2d,hum_2d,wind_speed_2d,weather_code_2d,is_holiday_2d,is_weekend_2d,season_2d
timestamp,,,,,,,,,
2017-01-03 20:00:00,911.973888,-0.391495,-0.436668,3.048523,2.393595,-1.458044,-0.001695,-0.005068,-0.001897
2017-01-03 21:00:00,605.976575,0.055816,0.092908,-0.498301,-0.299870,0.343039,0.002149,0.001592,0.002632
2017-01-03 22:00:00,-122.642353,-0.008554,-0.016567,0.032014,-0.969842,-0.280344,0.000430,-0.001865,0.000581
2017-01-03 23:00:00,-303.629568,-0.006461,-0.018128,0.370488,0.215228,0.135710,-0.001502,0.006860,-0.000488


In [30]:
#Reverse the transformation to get the real forecast with correct values
def reverse_transformation(bike_train, bike_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    bike_fc = bike_forecast.copy()
    columns = bike_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            bike_fc[str(col)+'_1d'] = (bike_train[col].iloc[-1]-bike_train[col].iloc[-2]) + bike_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        bike_fc[str(col)+'_forecast'] = bike_train[col].iloc[-1] + bike_fc[str(col)+'_1d'].cumsum()
    return bike_fc

In [33]:
bike_reverse = reverse_transformation(bike_train, bike_forecast, second_diff=True)        
bike_reverse.loc[:, ['cnt_forecast','t1_forecast','t2_forecast','hum_forecast','wind_speed_forecast',
                     'weather_code_forecast','is_holiday_forecast','is_weekend_forecast','season_forecast']]

,cnt_forecast,t1_forecast,t2_forecast,hum_forecast,wind_speed_forecast,weather_code_forecast,is_holiday_forecast,is_weekend_forecast,season_forecast
timestamp,,,,,,,,,
2017-01-03 20:00:00,775.973888,4.608505,0.563332,84.048523,18.393595,2.541956,-0.001695,-0.005068,2.998103
2017-01-03 21:00:00,1115.924352,4.272826,0.219573,86.598744,17.487320,2.426950,-0.001242,-0.008544,2.998838
2017-01-03 22:00:00,1333.232462,3.928593,-0.140754,89.180979,15.611203,2.031601,-0.000358,-0.013884,3.000154
2017-01-03 23:00:00,1246.911004,3.577900,-0.519209,92.133701,13.950314,1.771961,-0.000976,-0.012365,3.000981


In [44]:
#test the accuracy
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    mae = np.mean(np.abs(forecast - actual))    # MAE
    return({'rmse':rmse,'mape':mape,'mae': mae,})
def adjust(val, length= 6): return str(val).ljust(length)

#print the accuracy
print('Forecast Accuracy of: cnt')
accuracy_prod = forecast_accuracy(bike_reverse['cnt_forecast'].values, bike_test['cnt'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('Forecast Accuracy of: t1')
accuracy_prod = forecast_accuracy(bike_reverse['t1_forecast'].values, bike_test['t1'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: t2')
accuracy_prod = forecast_accuracy(bike_reverse['t2_forecast'].values, bike_test['t2'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: hum')
accuracy_prod = forecast_accuracy(bike_reverse['hum_forecast'].values, bike_test['hum'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: wind_speed')
accuracy_prod = forecast_accuracy(bike_reverse['wind_speed_forecast'].values, bike_test['wind_speed'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: weather_code')
accuracy_prod = forecast_accuracy(bike_reverse['weather_code_forecast'].values, bike_test['weather_code'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: is_holiday')
accuracy_prod = forecast_accuracy(bike_reverse['is_holiday_forecast'].values, bike_test['is_holiday'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: is_weekend')
accuracy_prod = forecast_accuracy(bike_reverse['is_weekend_forecast'].values, bike_test['is_weekend'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('Forecast Accuracy of: season')
accuracy_prod = forecast_accuracy(bike_reverse['season_forecast'].values, bike_test['season'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


Forecast Accuracy of: cnt
rmse   :  883.1477
mape   :  3.917
mae    :  807.7604
Forecast Accuracy of: t1
rmse   :  1.2401
mape   :  0.2179
mae    :  1.153
Forecast Accuracy of: t2
rmse   :  1.3067
mape   :  0.9758
mae    :  1.2193
Forecast Accuracy of: hum
rmse   :  11.2796
mape   :  0.1316
mae    :  10.1155
Forecast Accuracy of: wind_speed
rmse   :  6.4923
mape   :  0.2707
mae    :  6.1394
Forecast Accuracy of: weather_code
rmse   :  1.46
mape   :  0.341
mae    :  1.3069
Forecast Accuracy of: is_holiday
rmse   :  0.0012
mape   :  inf
mae    :  0.0011
Forecast Accuracy of: is_weekend
rmse   :  0.0105
mape   :  inf
mae    :  0.01
Forecast Accuracy of: season
rmse   :  0.0012
mape   :  0.0003
mae    :  0.001
